In [ ]:
!pip install transformers
!pip install datasets
!pip install ipywebrtc
!pip install soundfile
!pip install pydub
!pip install ffmpeg-python

# Automatic Speech Recognition

In [ ]:
import torch
from transformers import pipeline
from datasets import load_dataset
import torchaudio


In [ ]:


def get_device():
  device='cpu'
  if torch.cuda.is_available():
    device='cuda:0'
  elif torch.backends.mps.is_available():
    device='mps'
  else:
    device='cpu'
  return device

device = get_device()
print (device)

cuda:0


In [ ]:



pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  chunk_length_s=30,
  device=device,
)

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]

prediction = pipe(sample.copy(), batch_size=8)["text"]

# we can also return timestamps for the predictions
prediction = pipe(sample.copy(), batch_size=8, return_timestamps=True)["chunks"]


Found cached dataset librispeech_asr_dummy (/Users/premtimsina/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [ ]:
print(prediction)

[{'timestamp': (0.0, 5.44), 'text': ' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.'}]


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from ipywebrtc import CameraStream, AudioRecorder

# Create camera stream
camera = CameraStream(constraints={'audio': True, 'video': False})

# Create audio recorder
recorder = AudioRecorder(stream=camera)

# Display recorder
display(recorder)


AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
import ffmpeg
# Save the recording to a file
recorder.save('output.webm')
# Convert webm to wav
ffmpeg.input('output.webm').output('output.wav').run()

(None, None)

In [ ]:
import torchaudio
import torchaudio.transforms as T

waveform, sample_rate = torchaudio.load('output.wav')

# If audio is stereo, convert to mono
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0)

# Resample the waveform to 16kHz
resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)

# Squeeze the tensor to remove the channel dimension
waveform = waveform.squeeze()

# Convert tensor to numpy array
waveform_numpy = waveform.numpy()

In [ ]:
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  chunk_length_s=30,
  device=device,
)
prediction = pipe(waveform_numpy, batch_size=8)["text"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(prediction)

 I live in New York City. Today is very interesting day.
